In [1]:
from selenium import webdriver
from PIL import Image
import os
from os import path
import time
import random
import pandas as pd
import requests
import json
import numpy as np

In [1]:
import code.visualizations as viz
import code.data_preparation as dprep

In [2]:
api_keys = dprep.get_api_keys()
omdb_key = api_keys[0]
imdb_key = api_keys[1]
tmdb_key = api_keys[2]


## Scraping functions

### Scrape financial details from the numbers website and make a list of lists

In [4]:
financials_list = dprep.the_numbers_scraping()

### Make numbers_df, from the list of lists. Filter out corrupt/anomalous rows by rejecting movies with no worldwide takings and movies with budget < 1m dollars and only include movies with presence in the US

In [5]:
numbers_df = dprep.construct_prepare_numbers_df(financials_list)

### Make financials_df, which is a copy of numbers_df but with cleaned data, filtered for only last 15 years and empty column added to hold imdb_code


In [6]:
financials_15_df = dprep.make_fin_15(numbers_df)

## API functions

### Use omdb api to get attributes for movies in numbers_df. The response is stored in global dictionary omdb_attrs that will be used to create the attributes_df dataframe. It also updates the imdb_code column in financials_15_df. It also makes a list of failed calls, due to ambiguous title

In [7]:
(omdb_attrs, financials_15_df) = dprep.make_omdb_dict(financials_15_df, omdb_key)

C:\Users\james\Documents\flatiron_data_science\phase1\Project_fin\dsc-phase-1-project\code\data_preparation.py:224: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  financials_15_df['imdb_code'][financials_15_df.title == movie] = imdb_id


### The list of movies with failed calls is filtered to only include important movies -- stored in manual_list. Their imdb codes are found manually and stored in manual_imdb. They are zipped together and used to once again call the omdb api  and update the attributes dictionary (you can call omdb api either using title or imdb code) . 

In [ ]:
manual_list = ['Star Wars Ep. VII: The Force Awakens', 
'Star Wars Ep. VIII: The Last Jedi', 
'Prince of Persia: Sands of Time', 
'Fast and Furious 6', 
'The Chronicles of Narnia: The Voyage of the Daw…', 
'Harry Potter and the Deathly Hallows: Part II', 
'Harry Potter and the Deathly Hallows: Part I', 
'Fantastic Four: Rise of the Silver Surfer', 
'The Hangover 3', 
'Ford v. Ferrari', 
'Dr. Seuss’ The Grinch', 
'Mamma Mia: Here We Go Again!', 
'The Angry Birds Movie', 
'Wall Street 2: Money Never Sleeps',
'Dr. Seuss’ The Lorax', 
'All Eyez on Me', 
'John Wick: Chapter Two', 
'Underworld 3: Rise of the Lycans', 
'Halloween 2',        
'Precious (Based on the Novel Push by Sapphire)']

manual_imdb = ['tt2488496', 'tt2527336', 'tt0473075', 'tt1905041', 'tt0980970', 'tt1201607', 'tt0926084', 'tt0486576', 'tt1951261', 'tt1950186', 'tt2709692', 'tt6911608', 'tt1985949', 'tt1027718', 'tt1482459', 'tt1666185', 'tt4425200', 'tt0834001', 'tt1311067', 'tt0929632']

zipped_codes = list(zip(manual_list, manual_imdb))

In [8]:
(omdb_attrs, financials_15_df) = dprep.update_make_omdb_dict(financials_15_df, omdb_key, omdb_attrs, zipped_codes)

### Turn the attributes dict into a dataframe

In [1]:
attributes_df = dprep.make_attributes_df(omdb_attrs)

NameError: name 'dprep' is not defined

## Save the dataframes

In [10]:
dprep.save_files(financials_list, numbers_df, financials_15_df, omdb_attrs, attributes_df)

**DATAFRAMES** <br><br>
**numbers_df** - all financial data from The Numbers website<br>
**financials_15_df** - all financial data for numbers movies in last 15 years <br>
**financials_hits_df** - same data but filtered out loss making movies, those only succesful outside english speaking world and those with no documented figures <br>
**financials_flops_df** - same but filtered out profit making movies <br>
<br>